# Bayes's Classification Version1

## 下載套件

In [1]:
import numpy as np 
import pandas as pd
# Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
# Accuracy
from sklearn import metrics
# 選出前三高的機率
import heapq

PATH_in = "D:/NTPU/Project/8_Summary/" # 匯入路徑
PATH_out = "D:/NTPU/Project/8_Summary/" # 匯出路徑
file_name = "大福"

## 匯入檔案

In [2]:
# Icon Cut 已去停用詞 data
df = pd.read_excel(PATH_in+ "icon_nsw_c"+ file_name+ ".xlsx")
df = df[['ID', 'NoStopCut', 'emoji']]

# Icon Cut 詞頻, 用來建立 one-hot encode
all_words = pd.read_excel(PATH_in+ "icon_nsw_i"+ file_name+ ".xlsx")
all_words = list(all_words['Word'])

# Icon 對照表
all_icon = pd.read_excel(PATH_in + "all_icon.xlsx", sheet_name= "group")
df['Group'] = np.nan

## 刪除 Icon 不是我們想要的那些資料

我們只要分析那 200 多個 Icon, 所以其他的就刪掉, Group 就是那 90 類

△ 應該還可以再改進嗎......?

In [3]:
for i in range(df.shape[0]) :
    for s in range(all_icon.shape[0]) :
        if (df['emoji'][i] == all_icon['icon'][s]) :
            df['Group'][i] = all_icon['Group'][s]
            break

# 如果 Icon 不在那 90 類中, 就刪除
df = df[df['Group'].notna()] 

C:\Users\chenm\AppData\Local\Temp\ipykernel_11804\1311021683.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Group'][i] = all_icon['Group'][s]


## 把 list 打開

In [6]:
NoStopCut = []

for i in df.index :
    NoStopCut.append(eval(df['NoStopCut'][i]))

## One-hot encode

In [7]:
NoStopCut_oh = []

one_hot_encoded_df = pd.get_dummies(list(set(all_words)))
for i in range(len(NoStopCut)) :
    NoStopCut_oh.append(one_hot_encoded_df[NoStopCut[i]].T.sum().clip(0,1).values)

NoStopCut_oh = pd.DataFrame(NoStopCut_oh)
NoStopCut_oh.columns = all_words

In [8]:
NoStopCut_oh

,是,好,可愛,很,有,都,也,不,一,啊,...,烏龜,選區,補選,頭髪,剋制,笑慘,老闆娘,發自,吶,就緒
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8854,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8855,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## 把資料切成訓練與驗證

這邊之後要改成 k-fold

In [9]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(NoStopCut_oh, df[['Group']], test_size=0.3,random_state=109) # 70% training and 30% test

## GOGOGO

In [10]:
#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

D:\Anaconda3\envs\p2022_Project\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Accuracy?

In [27]:
Acc1 = metrics.accuracy_score(y_test, y_pred) # 這行只能執行一次, 如果執行第二次就會跑 Error, 但我不知道為何
Acc1

0.1380737396538751

## 抓前三高的類別

In [13]:
all_prob= gnb.predict_proba(X_test)
all_prob= pd.DataFrame(all_prob.T)

In [14]:
all_prob

,0,1,2,3,4,5,6,7,8,9,...,2648,2649,2650,2651,2652,2653,2654,2655,2656,2657
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.348239e-33
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.303709e-78
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.239643e-49
78,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.287857e-134
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.603844e-65
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.759911e-96


row 為類別, columms 為第幾筆資料, 我們總共有 90 類, 但這邊只有 81 row, 因為可能在訓練 model 時, 沒有全部的類別都被選進來, 也有可能是因為我的檔案裡本身就沒有這些類別的資料.  
下面會處理 row index 與類別號碼對不上的問題. 

In [17]:
group_sum = df['Group'].value_counts()
group_sum = group_sum.sort_index()

p1 = [] # 機率第一高
p2 = [] # 第二高
p3 = [] # 第三高

for i in range(all_prob.shape[1]) :
    test = list(all_prob[i])
    
    big3 = heapq.nlargest(3, test) # 前三高的數值
    big3_ind = list(map(test.index, big3)) # 前三高的位置
    p1.append(group_sum.index[big3_ind[0]])
    p2.append(group_sum.index[big3_ind[1]])
    p3.append(group_sum.index[big3_ind[2]])

prob3 = pd.DataFrame(zip(p1, p2, p3), columns= ['Prob1', 'Prob2', 'Prob3'])

In [18]:
prob3

,Prob1,Prob2,Prob3
0,17.0,41.0,1.0
1,77.0,1.0,1.0
2,58.0,1.0,1.0
3,87.0,1.0,1.0
4,42.0,1.0,1.0
...,...,...,...
2653,74.0,1.0,1.0
2654,42.0,1.0,1.0
2655,85.0,1.0,1.0
2656,20.0,1.0,1.0


△ 未解決  
    有些他其實只有算出一個機率, 只要那些 Prob2 = Prob3 的都是他其實所有類只有一個類有機率, 其他都是 0.  
    以及如果機率是 0, 還要被排進來嗎. -> 我覺得應該不能, 但程式還沒想好怎麼寫

In [19]:
y_test.reset_index(inplace = True) # test 真正的 Group

In [20]:
y_test

,index,Group
0,316,64.0
1,4787,77.0
2,6047,59.0
3,5987,74.0
4,7343,45.0
...,...,...
2653,8007,45.0
2654,7569,42.0
2655,8789,1.0
2656,131,39.0


我想看看選三個類別之後, 模型效果怎麼樣, 只要前三類有包含正確 Group 就為 True.

In [21]:
prob3['Correct'] = np.nan

for i in range(all_prob.shape[1]) :
    if y_test['Group'][i] in list(prob3.loc[i]) :
        prob3['Correct'][i] = True
    else:
        prob3['Correct'][i] = False

In [22]:
prob3

,Prob1,Prob2,Prob3,Correct
0,17.0,41.0,1.0,False
1,77.0,1.0,1.0,True
2,58.0,1.0,1.0,False
3,87.0,1.0,1.0,False
4,42.0,1.0,1.0,False
...,...,...,...,...
2653,74.0,1.0,1.0,False
2654,42.0,1.0,1.0,True
2655,85.0,1.0,1.0,True
2656,20.0,1.0,1.0,False


In [25]:
Acc2 = np.sum(prob3['Correct'] != 0) / all_prob.shape[1]
Acc2

0.15387509405568095

In [28]:
Acc2 - Acc1

0.015801354401805856

是有好一點啦

## 問題待解決

1. 跑一次要花超多時間
    首先在處理 icon 的部分就要花一些時間了, 在 one-hot encode 的部分尤其尤其費時.  
2. 停用詞
    我認為這個方法應該對停用詞還要更要求, 我們目前的停用詞還刪的不夠.
3. 未知詞
    還沒解決.QQ